# Classical Approach

In [22]:
import pandas as pd
from fetch import *
from collections import Counter
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.feature_extraction.text import TfidfVectorizer

## Load Data

In [7]:
df = fetch_data('function', 'mondo_names')

In [23]:
# Assuming df is your DataFrame
all_labels = [label for sublist in df['mondo_names'] for label in sublist]
label_counts = Counter(all_labels)
top_labels = [label for label, count in label_counts.most_common(10)]

mlb = MultiLabelBinarizer(classes=top_labels)
Y = mlb.fit_transform(df['mondo_names'])

# Convert 'function' column to a numeric format using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
X = tfidf_vectorizer.fit_transform(df['function'])

# Split data into training, validation, and testing sets
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.3, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)

/Users/danielmedina/Documents/Duke/classes/spring-2024/aipi540/baby-faces/env/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['12q14 microdeletion syndrome', '14q12 microdeletion syndrome', '16q24.3 microdeletion syndrome', '2-aminoadipic 2-oxoadipic aciduria', '2-hydroxyglutaric aciduria', '2-methylbutyryl-CoA dehydrogenase deficiency', '22q11.2 deletion syndrome', '2p21 microdeletion syndrome', '3-M syndrome', '3-hydroxy-3-methylglutaric aciduria', '3-hydroxy-3-methylglutaryl-CoA synthase deficiency', '3-hydroxyacyl-CoA dehydrogenase deficiency', '3-hydroxyisobutyryl-CoA hydrolase deficiency', '3-methylcrotonyl-CoA carboxylase 1 deficiency', '3-methylcrotonyl-CoA carboxylase deficiency', '3-methylglutaconic aciduria type 8', '3-methylglutaconic aciduria with cataracts, neurologic involvement and neutropenia', '3-phosphoglycerate dehydrogenase deficiency', '3M syndrome 3', '3MC syndrome', '3MC syndrome 1', '3MC syndrome 2', '3MC syndro

## Model Training

In [26]:
# Define and train the model
model = OneVsRestClassifier(SVC(kernel='linear', probability=True, class_weight='balanced'), n_jobs=1)
model.fit(X_train, Y_train)

OneVsRestClassifier(estimator=SVC(class_weight='balanced', kernel='linear',
                                  probability=True),
                    n_jobs=1)

## Evaluation

In [27]:
# Predict the labels on the validation set
Y_pred = model.predict(X_val)

# Calculate metrics
accuracy = accuracy_score(Y_val, Y_pred)
precision = precision_score(Y_val, Y_pred, average='samples')  # 'samples' gives a measure per sample
recall = recall_score(Y_val, Y_pred, average='samples')        # 'samples' or 'micro' could be used based on your specific needs
f1 = f1_score(Y_val, Y_pred, average='samples')                # Adjust 'average' as needed

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.4571013590621434
Precision: 0.6012692319726244
Recall: 0.6287879349270572
F1 Score: 0.5870945248597618


/Users/danielmedina/Documents/Duke/classes/spring-2024/aipi540/baby-faces/env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/danielmedina/Documents/Duke/classes/spring-2024/aipi540/baby-faces/env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/danielmedina/Documents/Duke/classes/spring-2024/aipi540/baby-faces/env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true 